In [1]:
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import yaml
import os
from openpyxl import load_workbook

/tmp/ipykernel_611268/1774324246.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
flag_project = 1 # 0--regular; 1--repartrition

In [3]:
if flag_project == 0:
    with open('a-project.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project/output_FJSJ_16-01/'               # Harmon server
elif flag_project == 1:
    with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project_repartrition/repartrition_01-03/'               # Harmon server

In [4]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  ./
dir_project_workspace:  ./
dir_project:  ./project_repartition_v4.0/output_repar_v9.2_01-01/


In [5]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()    

In [6]:
key_subworks = info_basic['key_subworks']
key_subworks

['1--08-03',
 '2--09-03',
 '3--10-03',
 '4--11-03',
 '5--12-03',
 '6--13-03',
 '7--14-03',
 '8--15-03',
 '9--16-03',
 '10--18-03',
 '11--19-03',
 '12--20-03',
 '13--21-03',
 '14--22-03',
 '15--23-03',
 '16--24-03',
 '17--25-03',
 '18--26-03',
 '19--27-03',
 '20--28-03',
 '21--29-03',
 '22--30-03',
 '23--31-03',
 '25--33-03',
 '26--34-03',
 '27--35-03',
 '28--36-03',
 '29--37-03',
 '30--41-03',
 '31--42-03',
 '33--46-03',
 '34--47-03',
 '35--48-03',
 '36--52-03',
 '37--53-03',
 '38--07-04',
 '39--08-04',
 '40--09-04',
 '41--10-04',
 '42--12-04',
 '43--13-04',
 '44--17-04',
 '45--20-04',
 '46--21-04',
 '48--23-04',
 '49--24-04',
 '50--25-04',
 '51--27-04',
 '52--28-04',
 '53--29-04',
 '54--30-04',
 '55--31-04',
 '56--32-04',
 '57--33-04',
 '58--34-04',
 '59--35-04',
 '60--36-04',
 '63--47-04',
 '65--49-04',
 '66--52-04',
 '67--53-04',
 '68--08-05',
 '69--12-05',
 '70--13-05',
 '71--14-05',
 '72--16-05',
 '75--22-05',
 '76--24-05',
 '77--26-05',
 '78--27-05',
 '79--28-05',
 '80--29-05',
 

In [7]:
dir_partition = dir_project+info_basic['dir_partition']

In [8]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
stalist_all = stainfo['Station'].tolist() 
lat_all =  stainfo['latitude'].tolist() 
lon_all =  stainfo['longitude'].tolist() 
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

In [9]:
#stalistname = dir_project+ info_basic['stalistname']
#stainfo_all = pd.read_excel(stalistname,sheet_name=None)

In [10]:
def calculate_centroid(coordinates):
    return np.mean(coordinates, axis=0)

In [11]:
def loc_key(key_subwork):
    global stalist_all
    global lat_all
    global lon_all
    key = 'R'+key_subwork[0:2]+key_subwork[3:]
    if key in stalist_all:
        lat_this = lat_all[stalist_all.index(key)]
        lon_this = lon_all[stalist_all.index(key)]
    else:
        key1 = 'R' + str(int(key_subwork[0:2])-1).zfill(2)+key_subwork[3:5]
        key2 = 'R' + str(int(key_subwork[0:2])+1).zfill(2)+key_subwork[3:5]
        indx1 = stalist_all.index(key1)
        indx2 = stalist_all.index(key2)
        lat_this= (lat_all[indx1]+lat_all[indx2])/2
        lon_this = (lon_all[indx1]+lon_all[indx2])/2
    return lon_this,lat_this

In [12]:
lon_centroid_all = {}
lat_centroid_all = {}

In [13]:
"""
for key_subwork in key_subworks:
    stainfo = stainfo_all[key_subwork]
    stalist = stainfo['Station'].tolist()
    lat = stainfo['latitude'].tolist()
    lon = stainfo['longitude'].tolist()
    lon_centroid,lat_centroid = calculate_centroid(np.array([lon,lat]).T)
    lon_centroid_all[key_subwork] = lon_centroid
    lat_centroid_all[key_subwork] = lat_centroid
"""

"\nfor key_subwork in key_subworks:\n    stainfo = stainfo_all[key_subwork]\n    stalist = stainfo['Station'].tolist()\n    lat = stainfo['latitude'].tolist()\n    lon = stainfo['longitude'].tolist()\n    lon_centroid,lat_centroid = calculate_centroid(np.array([lon,lat]).T)\n    lon_centroid_all[key_subwork] = lon_centroid\n    lat_centroid_all[key_subwork] = lat_centroid\n"

In [14]:
for key in key_subworks:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    lat = lat_stations_this.astype(float)
    lon = lon_stations_this.astype(float)
    lon_centroid,lat_centroid = calculate_centroid(np.array([lon,lat]).T)
    lon_centroid_all[key] = lon_centroid
    lat_centroid_all[key] = lat_centroid

### Write into xlsx file

In [15]:
"""
books = load_workbook(stalistname)
writers = pd.ExcelWriter(stalistname, engine='openpyxl')
writers.book = books
df=pd.DataFrame()
df['key_subwork'] = list(key_subworks)
df['lon_centroid'] = list(lon_centroid_all.values())
df['lat_centroid'] = list(lat_centroid_all.values())
df.to_excel(writers,sheet_name='location',index=False)
writers.save()
"""

"\nbooks = load_workbook(stalistname)\nwriters = pd.ExcelWriter(stalistname, engine='openpyxl')\nwriters.book = books\ndf=pd.DataFrame()\ndf['key_subwork'] = list(key_subworks)\ndf['lon_centroid'] = list(lon_centroid_all.values())\ndf['lat_centroid'] = list(lat_centroid_all.values())\ndf.to_excel(writers,sheet_name='location',index=False)\nwriters.save()\n"

In [16]:
filename = dir_project + 'subwork_location.xlsx'
df=pd.DataFrame()
df['key_subwork'] = [str(key_subwork) for key_subwork in key_subworks]
df['lon_centroid'] = list(lon_centroid_all.values())
df['lat_centroid'] = list(lat_centroid_all.values())
df.to_excel(filename,sheet_name='location',index=False)

In [17]:
dir_project

'./project_repartition_v4.0/output_repar_v9.2_01-01/'